## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [63]:
import geopandas as gpd
from shapely.geometry import LineString, Point
from folium.plugins import MarkerCluster

import folium

from src.data_aquisition.overpass import get_pois_near_path
from src.visualization.map import map_viz

## 1. Take hard-coded route as reference

There are some standard formats for saving geo routes:
- GeoJSON (commonly used with web libraries like leaflet and google maps)
- GPX (supported by GPS device and mobile apps, has elevation)
- KML (used by google, good for integration with google maps)
- csv (simple approach)
- SHP (popular for GIS)

In [74]:
kml_file_path = 'path.kml'
gdf = gpd.read_file(kml_file_path, driver='KML')

route_geom = gdf.geometry.iloc[0]


m = map_viz(route = route_geom, buffer=0.0002)
m

### 2. Find objects along the route and get their info

#### 2.1 Specific info

How to find the info?
1. Create a buffer zone around the route
2. Use a geospatial query to find objects within this buffer
3. Retrieve data about the found objects

We can use 
1. OpenStreetMap via Overpass API (free)
2. Google Places API (advantages: ratings, photos, more places, but not free)
3. Foursquare API

In [ ]:
pois_near_path = get_pois_near_path(route_geom, buffer_distance=20, max_distance=20)
pois_near_path = [el for el in pois_near_path if el['lat'] is not None and el['lon'] is not None and el['name'] is not None]

https://developers.google.com/maps/documentation/places/web-service/search-along-route

In [78]:
m = map_viz(route=route_geom, markers=pois_near_path)
m

In [46]:
from src.data_aquisition.wikipedia import fetch_wikipedia_article
import src.processing.text_translation as translate

In [ ]:
poi_names = [(poi['name']) for poi in pois_near_path]
poi_names_en = [translate.detect_and_translate_to_english(poi) for poi in poi_names]
poi_names_lang = [translate.detect_language(poi) for poi in poi_names]

get wikipedia articles about the POIs. Search in english, if no results, then on local language.

In [60]:
for i in range(len(pois_near_path)):
    if poi_names_lang[i] == 'en':
        summary = fetch_wikipedia_article(poi_names_en[i], language='en', return_summary=True)
    else:
        summary = fetch_wikipedia_article(poi_names[i], language=poi_names_lang[i], return_summary=True)
        summary = translate.detect_and_translate_to_english(summary)

    pois_near_path[i]['wiki'] = summary

In [67]:
pois_near_path_filtered = [poi for poi in pois_near_path if poi['wiki']!='']

In [79]:
m = map_viz(route=route_geom, markers=pois_near_path_filtered)
m

#### 2.2 General info

1. Use street names
2. Town areas
3. Town
4. Country

### 3. Utilize LLM to summarize information

### 4. Generate text-to-speech

1. https://elevenlabs.io/text-to-speech (1$ for 30min + commercial use)
2. aws polly (12months free, 1mil char / month, 5 mil char / month)